In [1]:
# default_exp 
from nbdev.showdoc import *
import numpy as np
import matplotlib.pyplot as plt
import torch
import FRED
from FRED.trainers import *
if torch.__version__[:4] == '1.13': # If using pytorch with MPS, use Apple silicon GPU acceleration
    device = torch.device("cuda" if torch.cuda.is_available() else 'mps' if torch.has_mps else "cpu")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", device)
%load_ext autoreload
%autoreload 2

Using device cuda


# 04c01 Trainer: Flow Neighbor Loss and Diffusion Map Regularization
> A copy of FRED, tailored with particular loss functions

In [ ]:
class FlowNeighborDiffMap(Trainer):
    def __init__(self, dataloader, device = device):
        self.vizfiz = [
            save_embedding_visualization,
            visualize_points
        ]
        self.loss_weights = {
            "reconstruction": 1,
            "diffusion map regularization": 1,
            "flow neighbor loss": 1e-4,
            "smoothness": 0,
        }
        self.FE = ManifoldFlowEmbedder(
            embedding_dimension=2,
            embedder_shape=[3, 4, 8, 4, 2],
            device=device,
            sigma=0.5,
            flow_strength=0.5,
            smoothness_grid=True,
        )
        self.dataloader = dataloader
        self.title = "Vanilla MFE"
        self.epochs_between_visualization = 10
        self.num_epochs = 100
        self.timestamp = datetime.datetime.now().isoformat()
        os.mkdir(f"visualizations/{self.timestamp}")
        self.optim = torch.optim.Adam(self.FE.parameters())
        self.scheduler = None
        # for plotting
        self.labels = dataloader.dataset.labels
        self.X = dataloader.dataset.X
        